## Imports

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, Flatten
from tensorflow.keras.callbacks import EarlyStopping

import mlflow.keras

2024-01-09 18:27:46.755052: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-09 18:27:48.089982: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


## GPU 

In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    print("Name:", gpu.name, "  Type:", gpu.device_type)

Name: /physical_device:GPU:0   Type: GPU


2024-01-09 18:27:50.257283: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-01-09 18:27:50.424539: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-01-09 18:27:50.424634: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.


## Data

In [3]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

11490434/11490434 [==============================] - 3s 0us/step


In [4]:
y_cat_test = to_categorical(y_test,10)

In [5]:
y_cat_train = to_categorical(y_train,10)

In [6]:
x_train = x_train/255
x_test = x_test/255

Reshape to include channel dimension (in this case, 1 channel)

In [7]:
x_train = x_train.reshape(60000, 28, 28, 1)

In [8]:
x_test = x_test.reshape(10000,28,28,1)

# Training the Model

### Model

In [9]:
model = Sequential()

# CONVOLUTIONAL LAYER
model.add(Conv2D(filters=32, kernel_size=(4,4),input_shape=(28, 28, 1), activation='relu',))
# POOLING LAYER
model.add(MaxPool2D(pool_size=(2, 2)))

# FLATTEN IMAGES FROM 28 by 28 to 764 BEFORE FINAL LAYER
model.add(Flatten())

# 128 NEURONS IN DENSE HIDDEN LAYER (YOU CAN CHANGE THIS NUMBER OF NEURONS)
model.add(Dense(128, activation='relu'))

# LAST LAYER IS THE CLASSIFIER, THUS 10 POSSIBLE CLASSES
model.add(Dense(10, activation='softmax'))

# https://keras.io/metrics/
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy']) # we can add in additional metrics https://keras.io/metrics/

2024-01-09 18:27:54.730565: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-01-09 18:27:54.730792: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-01-09 18:27:54.730851: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-01-09 18:27:56.717928: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-01-09 18:27:56.718069: I tensorflow/compile

In [10]:
early_stop = EarlyStopping(monitor='val_loss',patience=2)

## Train the Model

In [11]:
mlflow.set_experiment('MNIST with TensorFlow 2')
mlflow.keras.autolog()

2024/01/09 18:27:57 INFO mlflow.tracking.fluent: Experiment with name 'MNIST with TensorFlow 2' does not exist. Creating a new experiment.


In [12]:
model.fit(x_train,y_cat_train,epochs=4,validation_data=(x_test,y_cat_test),callbacks=[early_stop])

2024/01/09 18:27:58 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '2c33c94cbcc64b2e9d1800aaf1c51969', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current tensorflow workflow


Epoch 1/4


2024-01-09 18:28:04.494588: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8906
2024-01-09 18:28:07.329924: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7f652e450610 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-01-09 18:28:07.330010: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Quadro T2000 with Max-Q Design, Compute Capability 7.5
2024-01-09 18:28:07.395688: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-01-09 18:28:07.814633: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


1875/1875 [==============================] - 30s 11ms/step - loss: 0.1437 - accuracy: 0.9563 - val_loss: 0.0542 - val_accuracy: 0.9821
Epoch 2/4
1875/1875 [==============================] - 20s 11ms/step - loss: 0.0482 - accuracy: 0.9845 - val_loss: 0.0462 - val_accuracy: 0.9845
Epoch 3/4
1875/1875 [==============================] - 20s 11ms/step - loss: 0.0316 - accuracy: 0.9900 - val_loss: 0.0405 - val_accuracy: 0.9864
Epoch 4/4
1/1 [==============================] - 0s 312ms/step
INFO:tensorflow:Assets written to: /tmp/tmpdl181jb_/model/data/model/assets


INFO:tensorflow:Assets written to: /tmp/tmpdl181jb_/model/data/model/assets
2024/01/09 18:29:48 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/opt/conda/lib/python3.10/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils."
